Python Threat Hunting — Parsing JSON

JavaScript Object Notation (JSON) is a data format used to exchange data between machines. It is typically seen in web technologies because it is lightweight, easy to read and write for humans, and is supported by various programming and scripting languages. You have likely encountered JSON data before when browsing the web or interacting with web APIs.

In [ ]:
import json
import requests
import validators
import re
import ipaddress

# Maltiverse API endpoints
DOMAIN_URL = 'https://api.maltiverse.com/hostname/'
HASH_URL = 'https://api.maltiverse.com/sample/'
IP_URL = 'https://api.maltiverse.com/ip/'

API_KEY = 'PASTE_YOUR_API_KEY_HERE'

# define get_headers function as per Maltiverse API Key usage example: curl 'https://api.maltiverse.com/hostname/dophuot.net' -H 'Authorization: Bearer API_KEY'
def get_headers():
    headers = {
        'Content-Type': 'application/json',
    }
    if API_KEY:
        headers['Authorization'] = f'Bearer {API_KEY}'
    return headers

# check whether API key has been defined
def check_api_key():
    if not API_KEY or API_KEY.strip() == '':
        print("WARNING: No valid API key configured!")
        return False
    else:
        print(f"API key configured.")
        return True

# test API key with a simple request to a legitimate domain
def test_api_key():
    if not API_KEY:
        print("Cannot test API key - no key provided")
        return False
    
    print("Testing API key with a simple request...")
    try:
        response = requests.get(DOMAIN_URL + 'google.com', headers=get_headers(), timeout=10)
        print(f"   Response status: {response.status_code}")
        if response.status_code == 200:
            print("API key is working!")
            return True
        else:
            print(f"Unexpected response: {response.status_code}")
            return False
    except requests.RequestException as e:
        print(f"Network error during API test: {e}")
        return False

# check whether input ip is a valid ip
def is_valid_ip(ip_string):
    try:
        ip_string = ip_string.strip()
        if '.' not in ip_string or len(ip_string) > 15:
            return False, None
        
        ip_obj = ipaddress.ip_address(ip_string)
        if isinstance(ip_obj, ipaddress.IPv4Address):
            return True, 'IPv4'
        else:
            return False, None
    except (ipaddress.AddressValueError, ValueError):
        return False, None

# check input hash is a valid hash like MD5, SHA1, SHA256 as per their typical patterns
def is_valid_hash(hash_string):
    hash_string = hash_string.strip().lower()
    if len(hash_string) == 32 and re.match(r'^[a-f0-9]{32}$', hash_string):
        return True, 'MD5'
    elif len(hash_string) == 40 and re.match(r'^[a-f0-9]{40}$', hash_string):
        return True, 'SHA1'
    elif len(hash_string) == 64 and re.match(r'^[a-f0-9]{64}$', hash_string):
        return True, 'SHA256'
    else:
        return False, None

# check domain in Maltiverse
def check_domain(domain):
    try:
        response = requests.get(DOMAIN_URL + domain, headers=get_headers(), timeout=10)
        if response.status_code == 200:
            result = json.loads(response.text)
            try:
                classification = result['classification']
                print(f"\n=> The domain {domain} has been identified as {classification} by Maltiverse")
                
                # Additional information if available
                if 'score' in result:
                    print(f"   Score: {result['score']}")
                if 'tags' in result and result['tags']:
                    print(f"   Tags: {', '.join(result['tags'])}")
                    
            except KeyError:
                print(f"\n - The domain name {domain} cannot be classified by Maltiverse")
        else:
            print(f"\n - Error checking domain: HTTP {response.status_code}")
    except requests.RequestException as e:
        print(f"\n - Network error occurred: {e}")

# check hash in Maltiverse
def check_hash(file_hash, hash_type):
    try:
        response = requests.get(HASH_URL + file_hash, headers=get_headers(), timeout=10)
        if response.status_code == 200:
            result = json.loads(response.text)
            try:
                classification = result['classification']
                print(f"\n=> The {hash_type} hash {file_hash} has been identified as {classification} by Maltiverse")
                
                # Additional information if available
                if 'score' in result:
                    print(f"   Score: {result['score']}")
                if 'tags' in result and result['tags']:
                    print(f"   Tags: {', '.join(result['tags'])}")
                if 'file_names' in result and result['file_names']:
                    print(f"   Known filenames: {', '.join(result['file_names'][:3])}")  # Show first 3 filenames
                if 'file_size' in result:
                    print(f"   File size: {result['file_size']} bytes")
                    
            except KeyError:
                print(f"\n - The hash {file_hash} cannot be classified by Maltiverse")
        else:
            print(f"\n - Error checking hash: HTTP {response.status_code}")
    except requests.RequestException as e:
        print(f"\n - Network error occurred: {e}")

# check ip in Maltiverse
def check_ip(ip_address, ip_type):
    try:
        response = requests.get(IP_URL + ip_address, headers=get_headers(), timeout=10)
        if response.status_code == 200:
            result = json.loads(response.text)
            try:
                classification = result['classification']
                print(f"\n=> The {ip_type} address {ip_address} has been identified as {classification} by Maltiverse")
                
                # Additional information if available
                if 'score' in result:
                    print(f"   Score: {result['score']}")
                if 'tags' in result and result['tags']:
                    print(f"   Tags: {', '.join(result['tags'])}")
                if 'location' in result and result['location']:
                    location = result['location']
                    if 'country' in location:
                        print(f"   Country: {location['country']}")
                    if 'city' in location:
                        print(f"   City: {location['city']}")
                if 'as_name' in result:
                    print(f"   ASN: {result['as_name']}")
                if 'registrar' in result:
                    print(f"   Registrar: {result['registrar']}")
                    
            except KeyError:
                print(f"\n - The IP address {ip_address} cannot be classified by Maltiverse")
        else:
            print(f"\n - Error checking IP: HTTP {response.status_code}")
    except requests.RequestException as e:
        print(f"\n - Network error occurred: {e}")

# define the main function
def main():
    print("Maltiverse Domain, IP, and Hash Checker")
    print("=========================================")
    
    api_key_ok = check_api_key()
    
    if api_key_ok:
        if test_api_key():
            print("Ready to go!\n")
        else:
            print("API key test failed. Please check your key.\n")
            response = input("Continue anyway? (y/n): ").lower().strip()
            if response != 'y':
                print("Exiting...")
                return
    else:
        print()
        response = input("Continue without API key? (y/n): ").lower().strip()
        if response != 'y':
            print("Exiting...")
            return
    
    while True:
        user_input = input("Please enter domain, IP, or hash to search for (or type exit to quit): ")
        clean_input = user_input.strip()

        if clean_input.lower() == "exit":
            print("Goodbye!")
            break

        # Check if input is a valid hash
        if is_valid_hash(clean_input)[0]:
            is_valid, hash_type = is_valid_hash(clean_input)
            print(f"\nChecking {hash_type} hash: {clean_input}")
            check_hash(clean_input.lower(), hash_type)
            
        # Check if input is a valid IP
        elif is_valid_ip(clean_input)[0]:
            is_valid, ip_type = is_valid_ip(clean_input)
            print(f"\nChecking {ip_type} address: {clean_input}")
            check_ip(clean_input, ip_type)
            
        # Check if input is a valid domain
        elif validators.domain(clean_input):
            print(f"\nChecking domain: {clean_input}")
            check_domain(clean_input)
            
        else:
            print(f"\n - '{clean_input}' is not a valid domain, IP address, or hash. Please try again...")
        
        print()

if __name__ == "__main__":
    main()

Maltiverse Domain, IP, and Hash Checker
API key configured.
Testing API key with a simple request...
   Response status: 200
API key is working!
Ready to go!


Checking IPv4 address: 104.194.222.219

=> The IPv4 address 104.194.222.219 has been identified as neutral by Maltiverse


Checking IPv4 address: 104.223.120.159

=> The IPv4 address 104.223.120.159 has been identified as malicious by Maltiverse
   ASN: AS36352 HostPapa


Checking SHA256 hash: ec8952dc14bac73174cef02a489539e244b378b7de76c771126a8ba7ce532efd

=> The SHA256 hash ec8952dc14bac73174cef02a489539e244b378b7de76c771126a8ba7ce532efd has been identified as malicious by Maltiverse


Checking domain: devsetgroup.com

=> The domain devsetgroup.com has been identified as malicious by Maltiverse

Goodbye!
